In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objs as go
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV


# Классификация текстов с использованием Наивного Байесовского Классификатора

## Задание 1 (1 балл)

Откройте данные. Узнайте, сколько в них спам- и не спам-писем. Визуализируйте полученные соотношение подходящим образом.

In [2]:
df = pd.read_csv('spam_or_not_spam.csv')
display(df.head())
display(df.info())

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2999 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


None

In [3]:
# Подсчет количества элементов каждого класса в колонке 'label' с помощью метода value_counts()
counts = df['label'].value_counts()
# Подсчет процентного соотношения каждого класса в колонке 'label' с помощью метода value_counts() с параметром normalize=True
# и умножение на 100 для перевода в проценты
percentages = df['label'].value_counts(normalize=True) * 100
# Объединение Series counts и percentages в DataFrame result с помощью функции concat()
result = pd.concat([counts, percentages], axis=1)
# Задание названий столбцов 'Count' и 'Percentage' для DataFrame result
result.columns = ['Count', 'Percentage']
# Вывод DataFrame result на экран
print(result)

       Count  Percentage
label                   
0       2500   83.333333
1        500   16.666667


In [4]:
# Добавляем подписи для классов
class_labels = {0: 'Не спам', 1: 'Спам'}
# Получаем частоты классов
counts = df['label'].value_counts()
# Заменяем индексы (классы) на соответствующие им подписи
counts.index = counts.index.map(class_labels)
# Создаем объект для диаграммы
fig = go.Figure(data=[go.Pie(labels=counts.index, values=counts.values, hole=0.5)])
# Задаем название диаграммы
fig.update_layout(title="Распределение классов")
# Выводим диаграмму на экран
fig.show()

## Задание 2 (2 балла)

Вам необходимо предобработать ваши данные и перевести их в векторный вид. Подгрузим необходимый модуль:

>Вынесено в начало

Замените в данных все пустые строки и строки, состоящие из пробелов, на пропуски (NaN). После этого удалите из данных все строки, в которых наблюдаются пропущенные значения.

In [5]:
# Заменяем пустые строки и строки, состоящие из пробелов, на пропуски (NaN)
df.replace(['', ' '], np.nan, inplace=True)
# Удаляем все строки, содержащие пропущенные значения
df.dropna(inplace=True)
# Смотрим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2997 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2997 non-null   object
 1   label   2997 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 70.2+ KB


Переводим данные в векторный вид:

In [6]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["email"])

Определите, сколько теперь признаков в нашем наборе данных:

In [7]:
print(f'После обработки стало {X.shape[1]} признаков')

После обработки стало 34116 признаков


## Задание 3 (2 балла)

Определите целевую переменную и признаки:

In [8]:
# Определяем целевую переменную
y = df['label']
# выводим результат
print(f'Матрица X: {X.shape}')
print(f'Метки классов y: {y.shape}')

Матрица X: (2997, 34116)
Метки классов y: (2997,)


Разделите выборку на обучающую и тестовую, используя стратифицированное разбиение (параметр `stratify` установите в значение вектора ответов y) размер тестовой выборки (`test_size`) возьмите как 0.25, параметр `random_state` определите со значением 42:

In [9]:
# Разбиваем выборку на обучающую и тестовую, сохраняя пропорции классов
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
# выводим результат
print(f'Размер обучающей выборки: {X_train.shape}')
print(f'Размер тестовой выборки: {X_test.shape}')

Размер обучающей выборки: (2247, 34116)
Размер тестовой выборки: (750, 34116)


Рассчитайте среднее значение целевой переменной по тестовой выборке:

In [10]:
mean_y_test = round(y_test.mean(), 3)
print("Среднее значение целевой переменной по тестовой выборке:", mean_y_test)

Среднее значение целевой переменной по тестовой выборке: 0.165


## Задание 4 (3 балла)

Определите и обучите подходящий алгоритм с параметром alpha = 0.01

In [11]:
# Создаем объект классификатора
clf = MultinomialNB(alpha=0.01)
# Обучаем классификатор на обучающей выборке
clf.fit(X_train, y_train)

MultinomialNB(alpha=0.01)

Оцените результат с точки зрения всех известных вам метрик (не менее трёх):

In [12]:
# Вычисляем accuracy
accuracy = clf.score(X_test, y_test)
print("Accuracy:", round(accuracy, 3))
# Вычисляем precision
y_pred = clf.predict(X_test)
precision = precision_score(y_test, y_pred)
print("Precision:", round(precision, 3))
# Вычисляем recall
recall = recall_score(y_test, y_pred)
print("Recall:", round(recall, 3))
# Вычисляем F1-меру
f1 = f1_score(y_test, y_pred)
print("F1-мера:", round(f1, 3))

Accuracy: 0.987
Precision: 1.0
Recall: 0.919
F1-мера: 0.958


>Давайте проанализируем значения метрик, вычисленных для классификатора.

* Accuracy: 0.987. Эта метрика показывает, что доля правильных ответов классификатора на тестовой выборке составляет 0.987, или примерно 98.67%. То есть, из всех объектов, которые классификатор пытался классифицировать на тестовой выборке, он правильно определил 98.67% объектов.

* Precision: 1.0. Эта метрика показывает, что все объекты, которые классификатор отнес к положительному классу, были действительно положительными. Иными словами, если классификатор отнес объект к положительному классу, то можно быть уверенным, что этот объект действительно принадлежит к этому классу.

* Recall: 0.919. Эта метрика показывает, что классификатор правильно определил 91.94% всех объектов, которые принадлежат к положительному классу. Иными словами, если объект действительно принадлежит к положительному классу, то классификатор смог определить его с вероятностью 91.94%.

* F1-мера: 0.958. Это метрика, которая объединяет precision и recall в единую метрику. В данном случае, значение F1-меры 0.958 говорит о том, что классификатор достигает хорошего баланса между precision и recall, и его результаты можно считать достаточно точными.

>Таким образом, значения метрик свидетельствуют о том, что классификатор показал хорошие результаты на тестовой выборке, и его можно использовать для классификации новых объектов.

Нарисуйте ROC-кривую:

In [13]:
# Вычисляем значения TPR и FPR на тестовой выборке
fpr, tpr, thresholds = roc_curve(y_test, clf.predict_proba(X_test)[:, 1])
# Вычисляем значение AUC-ROC
auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
print("AUC-ROC:", auc)
# Строим ROC-кривую
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC-кривая (AUC-ROC = %0.2f)' % auc))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'), name='Случайная модель'))
fig.update_layout(xaxis_title='Ложно положительные решения', yaxis_title='Истинно положительные решения', title='ROC-кривая')
fig.show()

AUC-ROC: 0.9950659589817583


## Задание 5 (3 балла)

Переберите несколько значений alpha с помощью кросс-валидации. Оцените, зависит ли от этого параметра качество классификации.

In [14]:
# Задаем набор значений параметра alpha с шагом 0.05
param_grid = {'alpha': np.arange(0.01, 10, 0.05)}
# Создаем объект GridSearchCV
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5)
# Обучаем классификатор на обучающей выборке
grid_search.fit(X_train, y_train)
# Выводим наилучшее значение параметра alpha и соответствующее ему качество классификации
best_alpha = grid_search.best_params_['alpha']
best_score = grid_search.best_score_
print("Наилучшее значение параметра alpha:", best_alpha)
print("Качество классификации:", best_score)

# Получаем значения параметра alpha и соответствующие им значения F1-меры
alphas = [params['alpha'] for params in grid_search.cv_results_['params']]
f1_scores = grid_search.cv_results_['mean_test_score']
# Строим график зависимости значения F1-меры от значения параметра alpha
fig = go.Figure()
fig.add_trace(go.Scatter(x=alphas, y=f1_scores, mode='markers', marker=dict(color='blue'), name='F1-мера'))
# Добавляем отметку наилучшего значения alpha на график
fig.add_annotation(x=best_alpha, y=best_score, text='Наилучшее значение alpha', showarrow=True, arrowhead=1, font=dict(color='red'))
fig.update_layout(xaxis_title='Значение параметра alpha', yaxis_title='Значение F1-меры', title='Зависимость F1-меры от параметра alpha')
fig.show()

Наилучшее значение параметра alpha: 0.11
Качество классификации: 0.9915456570155902


>Можно сделать вывод, что оптимальное значение параметра alpha находится в интервале между 0.11 и 0.81.

>Такое поведение графика может быть связано с тем, что при значениях параметра alpha в интервале между 0.11 и 0.81 модель достигает наибольшего качества классификации на тестовой выборке. Однако, при значениях параметра alpha больше 0.81 модель начинает переобучаться и, как следствие, качество классификации начинает падать.